In [2]:
import librosa

In [3]:
import soundfile as sf

In [4]:
from scipy.signal import stft

In [5]:
from scipy.signal import resample

In [6]:
import scipy.io.wavfile as wavfile

In [7]:
import numpy as np
import scipy.signal as signal
import soundfile as sf
import matplotlib.pyplot as plt

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, BatchNormalization
from tensorflow.keras.activations import tanh, relu, sigmoid
import numpy as np


In [9]:
input_file = 'babble_8KHz.wav'
output_file = 'resampled_16KHz.wav'
target_sr = 16000

In [10]:

# Load the audio file
audio, sr = librosa.load(input_file, sr=None)

# Resample the audio to the target sample rate
resampled_audio = librosa.resample(audio,orig_sr= sr, target_sr=target_sr)

# Save the resampled audio to a new file
# librosa.output.write_wav(output_file,y= resampled_audio, sr=target_sr)
sf.write(output_file, resampled_audio, target_sr)

In [12]:
babble, fs_babble = sf.read("resampled_16KHz.wav")

In [13]:
babble.shape

(3763688,)

In [14]:
babble

array([-0.02706909, -0.04766846, -0.0557251 , ...,  0.03762817,
        0.03265381,  0.0149231 ])

In [20]:
# Initialization
Fs = 16000  # Sample frequency (Hz)
c = 340  # Sound velocity (m/s)
K = 256  # FFT length
M = 2  # Number of sensors
d = 0.2  # Inter sensor distance (m)
type_nf = 'spherical'  # Type of noise field: 'spherical' or 'cylindrical'
L = 14400  # Data length

In [15]:
data, Fs_data= sf.read("resampled_16KHz.wav")

In [16]:
data=data[:14400]

In [17]:
data.shape

(14400,)

In [18]:
Fs_data

16000

In [21]:
if Fs != Fs_data:
    raise ValueError('Sample frequency of input file is incorrect.')
data = data - np.mean(data)
babble = np.zeros((L, M))
for m in range(M):
    babble[:, m] = data[:]

In [22]:
# Generate matrix with desired spatial coherence
ww = Fs * np.arange(K // 2 + 1) / K
DC = np.zeros((M, M, K // 2 + 1))
for p in range(M):
    for q in range(M):
        if p == q:
            DC[p, q, :] = np.ones((1, 1, K // 2 + 1))
        else:
            if type_nf == 'spherical':
                DC[p, q, :] = np.sinc(ww * np.abs(p - q) * d / (c * np.pi))
            elif type_nf == 'cylindrical':
                DC[p, q, :] = signal.jn(0, ww * np.abs(p - q) * d / c)
            else:
                raise ValueError('Unknown noise field.')

In [23]:
len(DC[0][0])

129

In [24]:
# Compare desired and generated coherence
K_eval = 256
ww = 2 * np.pi * Fs * np.arange(K_eval // 2 + 1) / K_eval
sc_theory = np.zeros((M - 1, K_eval // 2 + 1))
sc_generated = np.zeros((M - 1, K_eval // 2 + 1))

In [25]:
# Generate sensor signals with desired spatial coherence
def mix_signals(babble, DC, method):
    L, M = babble.shape
    x = np.zeros((L, M-1))
    for k in range(K // 2 + 1):
        if method == 'cholesky':
            mat_t=np.array(DC[:,:,k])
            C = linalg.cholesky(mat_t)
        elif method == 'eigenvalue':
            eigenvalues, eigenvectors = np.linalg.eig(DC[:, :, k])
            eigenvalues = np.maximum(eigenvalues, 0)
            C = eigenvectors @ np.diag(np.sqrt(eigenvalues)) @ eigenvectors.T
        else:
            raise ValueError('Unknown method.')
        x_k = babble @ C.T
        x[:, :M-1] = x_k[:, :M-1]
    return x

In [26]:
x = mix_signals(babble, DC, 'eigenvalue')

In [27]:
x

array([[-0.02627196],
       [-0.04455214],
       [-0.05170172],
       ...,
       [ 0.00322005],
       [-0.00170882],
       [-0.00114011]])

In [28]:
x.shape

(14400, 1)

In [29]:
len(x)

14400

In [30]:
sum=0
xx=np.zeros(L)
xx[:]=x[:,0]
# for a in x:
#     xx
#     if(a[1]>0):
#         print("fds")
#         sum+=a[1]
print(sum)

0


In [31]:
xx.shape

(14400,)

In [32]:
window_length = 256
frame_shift = 128
padding_length = window_length - (14400 % window_length)
padded_xx = np.pad(xx, (0, padding_length), 'constant')
frequencies, time, X=stft(padded_xx, fs=16000, window='hann', nperseg=window_length)
# Print the shape of the STFT result
print("STFT shape:", X.shape)

STFT shape: (129, 115)


In [33]:
len(time)

115

In [34]:
X

array([[-5.25218726e-03+0.00000000e+00j, -1.17549176e-02+0.00000000e+00j,
         2.83644508e-03+0.00000000e+00j, ...,
         2.24710668e-02+0.00000000e+00j,  4.08709108e-03+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [-2.16113397e-03-7.25068759e-03j,  2.87110801e-03+3.55796617e-03j,
        -4.39967140e-03-4.34880163e-04j, ...,
        -1.36689659e-02+6.68728105e-03j,  1.49502530e-03-3.68358778e-03j,
         0.00000000e+00+0.00000000e+00j],
       [ 1.08425446e-02-2.92868741e-04j,  7.47955380e-03+5.34614973e-03j,
         3.73226026e-03+2.06304374e-03j, ...,
         3.27981206e-03-9.34120796e-03j, -2.67251109e-03-2.50668232e-03j,
         0.00000000e+00+0.00000000e+00j],
       ...,
       [-6.17108233e-05+2.13504981e-06j,  3.47889641e-07-3.55724738e-07j,
         1.98882122e-07-3.88802153e-07j, ...,
        -1.98624490e-06-3.51479458e-07j, -1.75824644e-06+8.57762057e-08j,
         0.00000000e+00+0.00000000e+00j],
       [ 6.19766194e-05-1.12116052e-06j, -6.

In [35]:
X.shape


(129, 115)

In [36]:
# function for SNR


def apply_snr(hrir,noise, snr):
    
    duration = 4.5 / 1000  # 4.5 ms

    # Define the original and target sampling rates
    original_fs = 1 / duration  # Sampling rate in Hz
    target_fs = 16000  # Target sampling rate in Hz

    # Calculate the number of samples in the original HRIR
    original_num_samples = len(hrir)

    # Calculate the number of samples in the target HRIR
    target_num_samples = int(original_num_samples * (target_fs / original_fs))

    # Resample the HRIR to the target sampling rate
    signal = resample(hrir, target_num_samples)

    # Load the WAV file
    #sample_rate, signal = wavfile.read(wav_file)

    # Calculate the noise energy
    signal_power = np.mean(signal ** 2)
    
    desired_noise_power = signal_power / (10**(snr / 10))

    # Scale the noise to the desired power
    current_noise_power = np.mean(noise**2)
    scaled_noise = noise * np.sqrt(desired_noise_power / current_noise_power)

    return scaled_noise

In [37]:
def apply_stft_hrir(hrir):
    
    duration = 4.5 / 1000  # 4.5 ms

    # Define the original and target sampling rates
    original_fs = 1 / duration  # Sampling rate in Hz
    target_fs = 16000  # Target sampling rate in Hz

    # Calculate the number of samples in the original HRIR
    original_num_samples = len(hrir)

    # Calculate the number of samples in the target HRIR
    target_num_samples = int(original_num_samples * (target_fs / original_fs))

    # Resample the HRIR to the target sampling rate
    resampled_hrir = resample(hrir, target_num_samples)
    
    #xx=resampled_hrir
    
    
    window_length = 256
    frame_shift = 128

    # Calculate the required padding length
    padding_length = window_length - (14400 % window_length)

    # Add padding to the time series array
    padded_time_series = np.pad(resampled_hrir, (0, padding_length), 'constant')

    # Apply STFT
    frequencies, times, Zxx = stft(padded_time_series, fs=16000, window='hann', nperseg=window_length, noverlap=window_length-frame_shift)

    # Print the shape of the STFT result
#     print("STFT shape:", Zxx.shape)
#     ret = np.arange(128 * 31).reshape(128, 31)
#     ret=Zxx[1:129][:31]
    list=[]
    for i in range(128):
        row = []
        for j in range(31):
            row.append(Zxx[i+1][j])
        list.append(row)
    return list

In [38]:
def apply_stft_noise(hrir,noise,snr):
    xx=apply_snr(hrir,noise,snr)
    
    window_length = 256
    frame_shift = 128
    padding_length = window_length - (14400 % window_length)
    padded_xx = np.pad(xx, (0, padding_length), 'constant')
    frequencies, time, X=stft(padded_xx, fs=16000, window='hann', nperseg=window_length)
    # Print the shape of the STFT result
#     print("STFT shape:", X.shape)
#     ret = np.arange(128 * 31).reshape(128, 31)
#     ret=X[1:129][0:31]
    list=[]
    for i in range(128):
        row = []
        for j in range(31):
            row.append(X[i+1][j])
        list.append(row)
    return list

    

In [39]:
import json

In [40]:
f = open('dictonary_DP_RTF.json')

In [41]:
jk=json.load(f)

In [42]:
type(jk['subject_003']['hrir_l'][0])

list

In [47]:
jk['subject_003'].keys()

dict_keys(['OnR', 'OnL', 'ITD', 'hrir_r', 'hrir_l', 'name'])

In [43]:
jk.keys()

dict_keys(['subject_003', 'subject_008', 'subject_009', 'subject_010', 'subject_011', 'subject_012', 'subject_015', 'subject_017', 'subject_018', 'subject_019', 'subject_020', 'subject_021', 'subject_027', 'subject_028', 'subject_033', 'subject_040', 'subject_044', 'subject_048', 'subject_050', 'subject_051', 'subject_058', 'subject_059', 'subject_060', 'subject_061', 'subject_065', 'subject_119', 'subject_124', 'subject_126', 'subject_127', 'subject_131', 'subject_133', 'subject_134', 'subject_135', 'subject_137', 'subject_147', 'subject_148', 'subject_152', 'subject_153', 'subject_154', 'subject_155', 'subject_156', 'subject_158', 'subject_162', 'subject_163', 'subject_165'])

In [44]:
training=['subject_010', 'subject_011', 'subject_012', 'subject_015', 'subject_017', 'subject_018',   'subject_028',  'subject_044', 'subject_048', 'subject_050', 'subject_051', 'subject_058', 'subject_059', 'subject_060',  'subject_124',  'subject_127', 'subject_134', 'subject_135', 'subject_137', 'subject_147', 'subject_148', 'subject_152', 'subject_153', 'subject_154', 'subject_155', 'subject_156', 'subject_158', 'subject_162', 'subject_163', 'subject_165']

In [45]:
validation=['subject_061', 'subject_065', 'subject_119','subject_126', 'subject_131', 'subject_133']

In [46]:
test=['subject_003', 'subject_008', 'subject_009', 'subject_019', 'subject_020', 'subject_021','subject_027','subject_033', 'subject_040']

In [49]:
len(validation)

6

In [50]:
type(jk['subject_028']['hrir_l'][0][0])

list

In [51]:
from collections.abc import Iterable
def get_dimensions(lst):
    dimensions = []
    while isinstance(lst, Iterable):
        dimensions.append(len(lst))
        lst = lst[0] if lst else None
    return dimensions

In [52]:
dc={}


In [55]:
#new DPRTF
def DPRTF_function_to_2d_list(list_1,list_2):
    new_list_3Fx1 = []
    for i in range(len(list_1)):
#         new_row=[]
        for j in range(len(list_1[0])):
            l1=apply_stft_hrir(list_1[i][j])
            l2=apply_stft_hrir(list_2[i][j])
            list_sum_1=[sum(row)/len(row) for row in l1]
            list_sum_2=[sum(row)/len(row) for row in l2]
            
            list_mag=[np.log10(np.abs(list_sum_2[i]/list_sum_1[i])) for i in range(len(list_sum_1))]
            list_pha=[np.angle(list_sum_2[i]/list_sum_1[i]) for i in range(len(list_sum_1))]
            
            list_sin=[np.sin(a) for a in list_pha]
            list_cos=[np.cos(a) for a in list_pha]
            
            list_fin=list_mag+list_sin+list_cos
            
            new_list_3Fx1.append(list_fin)
            
            
#         new_row = [apply_stft_hrir(element) for element in row]
#         new_r=new_row
#         print(len(new_r[0][0]))
#         print(len(new_r[0]))

#         new_list.append(new_row)
    return new_list_3Fx1

In [6]:
x=1+2j
y=2+1j
print(x/y)

(0.8+0.6j)


In [ ]:
#new hrir l r
def apply_function_to_2d_list(list_1,list_2):
    new_list_mag = []
    new_list_pha = []
    for i in range(len(list_1)):
#         new_row=[]
        for j in range(len(list_1[0])):
            l1=apply_stft_hrir(list_1[i][j])
            list_mag_1=[[np.log10(np.abs(l1[i][j])) for j in range(len(l1[0]))] for i in range(len(l1))]
            list_pha_1=[[np.angle(l1[i][j]) for j in range(len(l1[0]))] for i in range(len(l1))]
            
            l2=apply_stft_hrir(list_2[i][j])
            list_mag_2=[[np.log10(np.abs(l2[i][j])) for j in range(len(l2[0]))] for i in range(len(l2))]
            list_pha_2=[[np.angle(l2[i][j]) for j in range(len(l2[0]))] for i in range(len(l2))]
            m = len(list1)
            n = len(list1[0])
            list3 = []
            list4 = []
            for ii in range(m):
                row1 = []
                row2= []
                for jj in range(n):
                    row1.append([list_mag_1[ii][jj], list_mag_2[ii][jj]],[list_mag_1[ii][jj], list_mag_2[ii][jj]])
                    row2.append([list_pha_1[ii][jj], list_pha_2[ii][jj]],[list_mag_1[ii][jj], list_mag_2[ii][jj]])
                list3.append(row1)
                list4.append(row2)
            new_list_mag.append(list3)
            new_list_pha.append(list4)
            
            
#         new_row = [apply_stft_hrir(element) for element in row]
#         new_r=new_row
#         print(len(new_r[0][0]))
#         print(len(new_r[0]))

#         new_list.append(new_row)
    return [new_list_mag,new_list_pha]

In [53]:
# for a in ['subject_003']:
#     hrtf_r=apply_stft_hrir(jk[a]['hrir_r'][0][0])[1:][:31]
#     hrtf_l=apply_stft_hrir(jk[a]['hrir_l'][0][0])[1:][:31]
#     dc[a]=[hrtf_l,hrtf_r]

In [56]:
for a in ['subject_003']:
    fin=apply_function_to_2d_list(jk[a]['hrir_l'],jk[a]['hrir_r'])
#     hrtf_l=apply_function_to_2d_list()

#     for i in range(hrtf_r):
#         for j in range (hrtf_r[0]):
            
        
    dc[a]=fin

In [60]:
main_dc={}
t_dc={}
v_dc={}
test_dc={}

In [ ]:
print('training')
for a in training:
    fin=apply_function_to_2d_list(jk[a]['hrir_l'],jk[a]['hrir_r'])
    list_test=DPRTF_function_to_2d_list(jk[a]['hrir_l'],jk[a]['hrir_r'])
    print(a)
    t_dc['mag']=t_dc['mag']+fin[0]
    t_dc['pha']=t_dc['pha']+fin[1]
    t_dc['test']=t_dc['test']+list_test
main_dc['training']=t_dc
print('validation')
for a in validation:
    fin=apply_function_to_2d_list(jk[a]['hrir_l'],jk[a]['hrir_r'])
    list_test=DPRTF_function_to_2d_list(jk[a]['hrir_l'],jk[a]['hrir_r'])
    print(a)
    v_dc['mag']=v_dc['mag']+fin[0]
    v_dc['pha']=v_dc['pha']+fin[1]
    v_dc['test']=v_dc['test']+list_test
main_dc['validation']=v_dc
print('testing')
for a in test:
    fin=apply_function_to_2d_list(jk[a]['hrir_l'],jk[a]['hrir_r'])
    list_test=DPRTF_function_to_2d_list(jk[a]['hrir_l'],jk[a]['hrir_r'])
    print(a)
    test_dc['mag']=test_dc['mag']+fin[0]
    test_dc['pha']=test_dc['pha']+fin[1]
    test_dc['test']=test_dc['test']+list_test
main_dc['testing']=test_dc

training
subject_010
subject_011
subject_012
subject_015
subject_017
subject_018
subject_028
subject_044
subject_048
subject_050
subject_051
subject_058
subject_059
subject_060
subject_124
subject_127
subject_134
subject_135
subject_137
subject_147
subject_148
subject_152
subject_153
subject_154
subject_155
subject_156


{}

In [1]:
import numpy as np

x = 1 + 2j

theta = np.angle(x)
magnitude = np.abs(x)

print("Theta:", theta)
print("Magnitude:", magnitude)

Theta: 1.1071487177940904
Magnitude: 2.23606797749979


In [4]:
main_dc


{}

In [58]:
print(len(dc['subject_003']))
print(len(dc['subject_003'][0]))
print(len(dc['subject_003'][0][0]))
print(len(dc['subject_003'][0][0][0]))
print(len(dc['subject_003'][0][0][0][0]))
print(type(dc))
print(type(dc['subject_003']))
print(type(dc['subject_003'][0]))
print(type(dc['subject_003'][0][0]))
print(type(dc['subject_003'][0][0][0]))
print(type(dc['subject_003'][0][0][0][0]))

25
50
128
31
2
<class 'dict'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [59]:
get_dimensions(dc['subject_003'])

[25, 50, 128, 31, 2]

In [310]:
# def apply_function_to_2d_list(list):
#     new_list = []
#     for row in list:
#         new_row = [apply_stft_hrir(element) for element in row]
#         new_r=new_row
# #         print(len(new_r[0][0]))
# #         print(len(new_r[0]))
#         new_list.append(new_r)
#     return new_list

In [146]:
jk["subject_003"].keys()

dict_keys(['OnR', 'OnL', 'ITD', 'hrir_r', 'hrir_l', 'name'])

In [15]:
#building CNN

In [140]:
# # Input layers
# input_mag = Input(shape=(F, N, 2))  # Magnitude spectrogram input
# input_phase = Input(shape=(F, N, 2))  # Phase spectrogram input

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, Activation, BatchNormalization
from keras.activations import tanh, relu, sigmoid
import numpy as np

In [16]:
# Assuming the input shape for each frame is FxNx2
F = 128  # Number of frequency bins
N = 31  # Number of time frames

In [17]:
# Input layers
input_mag = Input(shape=(F, N, 4))  # Magnitude spectrogram input
input_phase = Input(shape=(F, N, 4))  # Phase spectrogram input

In [18]:
# Magnitude branch
mag_conv1 = Conv2D(32, (1, 1), activation=tanh)(input_mag)
mag_conv2 = Conv2D(64, (3, 3))(mag_conv1)
mag_bn = BatchNormalization()(mag_conv2)
mag_relu = Activation(relu)(mag_bn)

In [ ]:
# Phase branch
phase_conv1 = Conv2D(32, (1, 1))(input_phase)
phase_activation_sin = Activation(lambda x: np.sin(x))(phase_conv1)
phase_activation_cos = Activation(lambda x: np.cos(x))(phase_conv1)
phase_conv2 = Conv2D(64, (3, 3))(np.concatenate(phase_activation_sin,phase_activation_cos,axis=-1)
phase_bn = BatchNormalization()(phase_conv2)
phase_relu = Activation(relu)(phase_bn)


# layers.Conv2D(32, (1, 1), input_shape=(128, 31, 4)),
#     layers.Lambda(lambda x: keras.backend.sin(x), output_shape=(128, 31, 32)),
#     layers.Lambda(lambda x: keras.backend.cos(x), output_shape=(128, 31, 32)),
#     layers.Concatenate(axis=-1),
#     layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
#     layers.BatchNormalization()

In [20]:
# Mask/weight branch
mask_conv1 = Conv2D(64, (3, 3))(input_mag)
mask_bn1 = BatchNormalization()(mask_conv1)
mask_relu1 = Activation(relu)(mask_bn1)
mask_conv2 = Conv2D(64, (3, 3))(mask_relu1)
mask_bn2 = BatchNormalization()(mask_conv2)
mask_sigmoid = Activation(sigmoid)(mask_bn2)

In [ ]:
# Multiply the learned magnitude and phase features with the mask
mag_masked = mag_relu * mask_sigmoid
phase_masked = phase_relu * mask_sigmoid

In [ ]:
# Create separate models for each branch
magnitude_model = Model(inputs=input_mag, outputs=mag_masked)
phase_model = Model(inputs=input_phase, outputs=phase_masked)
mask_model = Model(inputs=input_mag, outputs=mask_sigmoid)

In [ ]:
# Print the summary of the magnitude branch model
magnitude_model.summary()

In [ ]:
#CRNN 
from keras.layers import Concatenate, MaxPooling2D, GRU, Dense

# Concatenate magnitude and phase features
concatenated_features = Concatenate(axis=-1)([mag_masked, phase_masked])

# Convolutional modules
for _ in range(5):
    concatenated_features = Conv2D(64, (3, 3), activation=relu)(concatenated_features)
    concatenated_features = BatchNormalization()(concatenated_features)
    concatenated_features = Conv2D(64, (3, 3), activation=relu)(concatenated_features)
    concatenated_features = BatchNormalization()(concatenated_features)

    # Max pooling to compress the frequency dimension
    concatenated_features = MaxPooling2D(pool_size=(2,1))(concatenated_features)

# Reshape to prepare for GRU
reshaped_features = tf.keras.layers.Reshape(target_shape=(2*F,N))(concatenated_features)

# Gated Recurrent Unit (GRU)
gru_output = GRU(256, return_sequences=False)(reshaped_features)

# Fully-connected layer for DP-RTF prediction
fc_output = Dense(F, activation=tanh)(gru_output)

# Activation to fit DP-RTF elements into the range from -1 to 1
output = Activation(tanh)(fc_output)

# Create the final model
model = Model(inputs=[input_mag, input_phase], outputs=output)

# Print the summary of the model
model.summary()








In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Concatenate, MaxPooling2D, Reshape, GRU, Dense
from tensorflow.keras.activations import tanh, relu, sigmoid

# Define the model architecture
def create_model(F, N):
    # Input layers
    input_mag = Input(shape=(F, N, 4))  # Magnitude spectrogram input
    input_phase = Input(shape=(F, N, 4))  # Phase spectrogram input

    # Magnitude branch
    mag_conv1 = Conv2D(32, (1, 1), activation=tanh)(input_mag)
    mag_conv2 = Conv2D(64, (3, 3))(mag_conv1)
    mag_bn = BatchNormalization()(mag_conv2)
    mag_relu = Activation(relu)(mag_bn)

    # Phase branch
    phase_conv1 = Conv2D(32, (1, 1))(input_phase)
    phase_activation_sin = Activation(lambda x: np.sin(x))(phase_conv1)
    phase_activation_cos = Activation(lambda x: np.cos(x))(phase_conv1)
    phase_concat = Concatenate(axis=-1)([phase_activation_sin, phase_activation_cos])
    phase_conv2 = Conv2D(64, (3, 3))(phase_concat)
    phase_bn = BatchNormalization()(phase_conv2)
    phase_relu = Activation(relu)(phase_bn)

    # Mask/weight branch
    mask_conv1 = Conv2D(64, (3, 3))(input_mag)
    mask_bn1 = BatchNormalization()(mask_conv1)
    mask_relu1 = Activation(relu)(mask_bn1)
    mask_conv2 = Conv2D(64, (3, 3))(mask_relu1)
    mask_bn2 = BatchNormalization()(mask_conv2)
    mask_sigmoid = Activation(sigmoid)(mask_bn2)

    # Multiply the learned magnitude and phase features with the mask
    mag_masked = tf.keras.layers.Multiply()([mag_relu, mask_sigmoid])
    phase_masked = tf.keras.layers.Multiply()([phase_relu, mask_sigmoid])

    # Concatenate magnitude and phase features
    concatenated_features = Concatenate(axis=-1)([mag_masked, phase_masked])

    # Convolutional modules
    for _ in range(5):
        concatenated_features = Conv2D(64, (3, 3), activation=relu)(concatenated_features)
        concatenated_features = BatchNormalization()(concatenated_features)
        concatenated_features = Conv2D(64, (3, 3), activation=relu)(concatenated_features)
        concatenated_features = BatchNormalization()(concatenated_features)
        concatenated_features = MaxPooling2D(pool_size=(2, 1))(concatenated_features)

    # Reshape to prepare for GRU
    reshaped_features = Reshape(target_shape=(2 * F, N))(concatenated_features)

    # Gated Recurrent Unit (GRU)
    gru_output = GRU(256, return_sequences=False)(reshaped_features)

    # Fully-connected layer for DP-RTF prediction
    fc_output = Dense(F, activation=tanh)(gru_output)

    # Create the model
    model = tf.keras.Model(inputs=[input_mag, input_phase], outputs=fc_output)
    return model

# Create the model
model = create_model(F, N)  # Replace F and N with appropriate values

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Generate dummy data for training and testing
input_mag_train = np.random.rand(100, F, N, 4)  # Replace 100 with the actual training data size
input_phase_train = np.random.rand(100, F, N, 4)
output_train = np.random.rand(100, F)  # Replace 100 with the actual training data size

input_mag_test = np.random.rand(20, F, N, 4)  # Replace 20 with the actual test data size
input_phase_test = np.random.rand(20, F, N, 4)
output_test = np.random.rand(20, F)  # Replace 20 with the actual test data size

# Train the model
model.fit([input_mag_train, input_phase_train], output_train, epochs=10, batch_size=32)

# Test the model
loss = model.evaluate([input_mag_test, input_phase_test], output_test)
print('Test loss:', loss)
